In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
import numpy as np 
import random
from numpy import savetxt
from numpy import loadtxt
import pandas as pd 
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import os
import time
import re
import glob
import cv2
from zipfile import ZipFile
if(not os.path.exists("/content/gdrive/MyDrive/Images")):
  with ZipFile('/content/gdrive/MyDrive/NNDataset.zip','r') as zipObj:
    zipObj.extractall('/content/gdrive/MyDrive/Images')
if(not os.path.exists("/content/gdrive/MyDrive/testImages")):
  with ZipFile('/content/gdrive/MyDrive/data.zip','r') as zipObj:
    zipObj.extractall('/content/gdrive/MyDrive/testImages')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# augmantation
img_size=227
def augmantation(train_dir,img,label):
  path =  os.path.join(train_dir,img)
  image = cv2.imread(path)
  image = cv2.resize(image,(img_size,img_size))  
  image = image.reshape(img_size,img_size,3)  
  # Main
  train_images.append([np.array(image,dtype=object)])
  train_labels.append(label)
  # T1
  im1 = tf.image.flip_left_right(image)
  train_images.append([np.array(im1,dtype=object)])
  train_labels.append(label)
  # T2
  im2 = tf.image.flip_up_down(image)
  train_images.append([np.array(im2,dtype=object)])
  train_labels.append(label)
  # T3
  im3 = tf.image.flip_left_right(image)
  im4 = tf.image.flip_up_down(im3)
  train_images.append([np.array(im4,dtype=object)])
  train_labels.append(label)
  

In [ ]:
#prepairing train_images & train_labels
train_dir = "/content/gdrive/MyDrive/Images/Train"
train_images = [] 
train_labels =[]
classesCount = [0,0,0,0,0,0]
if (os.path.exists('/content/gdrive/MyDrive/pre_saved_files/train_images.npy')):  
  train_images = np.load('/content/gdrive/MyDrive/pre_saved_files/train_images.npy')
  train_labels = np.load('/content/gdrive/MyDrive/pre_saved_files/train_labels.npy')
else:
  for img in os.listdir(train_dir):
    name = img.split('_')[0]
    if(name == 'Basketball' and classesCount[0]<180 ):
      classesCount[0] = classesCount[0]+1
      augmantation(train_dir,img,[0])
    if(name == 'Football'and classesCount[1]<180):
      classesCount[1] = classesCount[1]+1
      augmantation(train_dir,img,[1])
    if(name == 'Rowing' and classesCount[2]<180):
      classesCount[2] = classesCount[2]+1
      augmantation(train_dir,img,[2])
    if(name == 'Swimming' and classesCount[3]<180):
      classesCount[3] = classesCount[3]+1
      augmantation(train_dir,img,[3])
    if(name == 'Tennis' and classesCount[4]<180):
      classesCount[4] = classesCount[4]+1
      augmantation(train_dir,img,[4])
    if(name == 'Yoga' and classesCount[5]<180):
      classesCount[5] = classesCount[5]+1
      augmantation(train_dir,img,[5])
  temp = list(zip(train_images, train_labels))
  random.shuffle(temp)
  train_images, train_labels = zip(*temp)
  test_images = train_images[:700]
  test_labels = train_labels[:700]
  train_images = np.array([i for i in train_images[700:]]).reshape(-1,img_size,img_size,3)
  train_labels = np.array([i for i in train_labels[700:]])
  train_images = np.asarray(train_images).astype('uint8')
  train_labels = np.asarray(train_labels).astype('uint8')
  np.save('/content/gdrive/MyDrive/pre_saved_files/train_images.npy', train_images)  
  np.save('/content/gdrive/MyDrive/pre_saved_files/train_labels.npy', train_labels)
print(train_images.shape)
print(train_labels.shape)  


(3620, 227, 227, 3)
(3620, 1)


In [ ]:
# for prepairing test_images & test_labels

# test_images = []
# test_labels = []
if (os.path.exists('/content/gdrive/MyDrive/pre_saved_files/test_images.npy')):  
  test_images = np.load('/content/gdrive/MyDrive/pre_saved_files/test_images.npy')
  test_labels = np.load('/content/gdrive/MyDrive/pre_saved_files/test_labels.npy')
else:  
  test_images = np.array([i for i in test_images]).reshape(-1,img_size,img_size,3)
  test_labels = np.array([i for i in test_labels])
  test_images = np.asarray(test_images).astype('uint8')
  test_labels = np.asarray(test_labels).astype('uint8')
  np.save('/content/gdrive/MyDrive/pre_saved_files/test_images.npy', test_images)  
  np.save('/content/gdrive/MyDrive/pre_saved_files/test_labels.npy', test_labels)
print(test_images.shape)
print(test_labels.shape)  

(700, 227, 227, 3)
(700, 1)


In [ ]:
train_ds=tf.data.Dataset.from_tensor_slices((train_images,train_labels))
test_ds=tf.data.Dataset.from_tensor_slices((test_images,test_labels))
train_ds_size=tf.data.experimental.cardinality(train_ds).numpy()
test_ds_size=tf.data.experimental.cardinality(test_ds).numpy()
print('Train size:',train_ds_size)
print('Test size:',test_ds_size)

Train size: 3620
Test size: 700


In [ ]:
train_ds=(train_ds          
           .shuffle(buffer_size=train_ds_size)
           .batch(batch_size=32,drop_remainder=True)
         )
test_ds=(test_ds         
          .shuffle(buffer_size=test_ds_size)
          .batch(batch_size=32,drop_remainder=True)
         )

In [ ]:
if (os.path.exists('/content/gdrive/MyDrive/pre_saved_files/saved_model7')):  
  Loaded_model = keras.models.load_model("/content/gdrive/MyDrive/pre_saved_files/saved_model7")
else:
  model=keras.models.Sequential([
      keras.layers.Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(img_size,img_size,3)),
      keras.layers.BatchNormalization(),
      keras.layers.MaxPool2D(pool_size=(2,2)),
      keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
      keras.layers.BatchNormalization(),
      keras.layers.MaxPool2D(pool_size=(3,3)),
      keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
      keras.layers.BatchNormalization(),
      keras.layers.Conv2D(filters=384, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
      keras.layers.BatchNormalization(),
      keras.layers.Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
      keras.layers.BatchNormalization(),
      keras.layers.MaxPool2D(pool_size=(2,2)),
      keras.layers.Flatten(),
      keras.layers.Dense(4096,activation='relu'),
      keras.layers.Dropout(0.5),
      keras.layers.Dense(4096,activation='relu'),
      keras.layers.Dropout(0.5),
      keras.layers.Dense(6,activation='softmax')   
  ])
  model.compile(
      loss='sparse_categorical_crossentropy',
      optimizer=tf.optimizers.SGD(lr=0.001),
      metrics=['accuracy']    
  )
  model.summary() 
  history=model.fit(
      train_ds,
      epochs=50,
      validation_data=test_ds,
      validation_freq=1,     
      )
  
  model.save('/content/gdrive/MyDrive/pre_saved_files/saved_model7',save_format="h5")

In [ ]:
loss, acc = Loaded_model.evaluate(test_images, test_labels, verbose=2)
print("accuracy: {:.2f}%".format(100 * acc))

22/22 - 1s - loss: 0.3725 - accuracy: 0.8757 - 797ms/epoch - 36ms/step
accuracy: 87.57%


In [ ]:
test_dir = "/content/gdrive/MyDrive/Images/Test"
test_images_names=[]
test_images2 = []
if (os.path.exists('/content/gdrive/MyDrive/pre_saved_files/test_images2.npy')):  
  test_images2 = np.load('/content/gdrive/MyDrive/pre_saved_files/test_images2.npy')
  for img in os.listdir(test_dir):
    test_images_names.append(img)
else:
  for img in os.listdir(test_dir):
    test_images_names.append(img)
    path =  os.path.join(test_dir,img)
    image = cv2.imread(path)
    image = cv2.resize(image,(img_size,img_size)) 
    test_images2.append([np.array(image,dtype=object)])    
  test_images2 = np.array([i for i in test_images2]).reshape(-1,img_size,img_size,3)
  test_images2 = np.asarray(test_images2).astype('uint8')
  np.save('/content/gdrive/MyDrive/pre_saved_files/test_images2.npy', test_images2)  
print(test_images2.shape)

(688, 227, 227, 3)


In [ ]:
predicted =Loaded_model.predict(test_images2)

22/22 [==============================] - 1s 24ms/step


In [ ]:
classes = [0,1,2,3,4,5]
CLASS_NAMES= ['Basketball', 'Football', 'Rowing', 'Swimming', 'Tennis', 'Yoga']
data =[]
for i in range(len(predicted)):
  mxindex = -1
  mx =0
  for j in range(6):
    if(predicted[i][j] > mx):
      mxindex = j
      mx = predicted[i][j]
  data.append([test_images_names[i],classes[mxindex]])
Datapd=pd.DataFrame(data)
Datapd.columns=['image_name','Label']
Datapd.to_csv('predictions.csv',index=False)
